# [실습] LangChain Tool Call과 Agent

LangChain의 기본 코드를 통해, Tool Call과 Agent가 작동하는 과정에 대해 알아보겠습니다.   

기본 라이브러리를 설치합니다.

In [ ]:
!pip install -U langchain_google_genai langchain_core langchain_community tavily-python

Google API Key를 설정합니다.

구글 로그인 후, https://aistudio.google.com/apikey 에서 발급받을 수 있습니다.

In [ ]:
import os

os.environ['GOOGLE_API_KEY'] = ''

Gemini 모델을 설정합니다.   
무료 API의 경우 분당 제한이 존재하므로, 랭체인의 Rate Limiter를 적용할 수 있습니다.

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_openai import ChatOpenAI

# Gemini API는 분당 10개 요청으로 제한
# 즉, 초당 약 0.167개 요청 (10/60)
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

# rate limiter를 LLM에 적용
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    rate_limiter=rate_limiter,
    # temperature
    # max_tokens
)

In [ ]:
llm.invoke("안녕? 너는 이름이 뭐니? 한 문장으로 말해줘.")

`invoke()`를 통해 llm에 입력을 전달합니다.    
랭체인 기본 클래스인 `Message` 계열 클래스를 직접 만들어 리스트로 전달하거나,   
프롬프트 템플릿을 통해 입력의 구성을 만들어 전달할 수 있습니다.

In [ ]:
# 1. HumanMessage를 이용한 방법

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

system_msg = SystemMessage(content="""사용자의 아래 [질문]에 대해 친절하게 답변하세요.
답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.""")
msg = HumanMessage(content="안녕! 너는 이름이 뭐니?")

messages = [system_msg, msg]

response = llm.invoke(messages)
response

Chat 모델의 출력은 AIMessage라는 형식으로 변환됩니다.

In [ ]:
print(response.content)

In [ ]:
# 2. ChatPromptTemplate을 이용한 방법

from langchain.prompts import ChatPromptTemplate

# 입력 변수가 {question}인 템플릿
prompt = ChatPromptTemplate([
    ('system', '''사용자의 아래 [질문]에 대해 친절하게 답변하세요.
     답변의 길이는 5문장을 넘지 않도록 하고, 개조식으로 작성하세요.'''),
     ('user', '''[질문]: {question}''')])
prompt


프롬프트와 llm을 |로 연결하여 체인을 구성합니다.   
(랭체인 기본 문법으로, LangChain 기초 복습 자료에서 자세히 보실 수 있습니다!)

In [ ]:
chain = prompt | llm
chain

In [ ]:
response = chain.invoke({'question':"랭체인과 랭그래프의 차이가 뭐야?"})

print(response.content)

In [ ]:
# 입력 변수가 1개일 때는 값만 invoke해도 됨

response = chain.invoke("패스트캠퍼스 랭그래프 강의 이름이 뭐야?")

print(response.content)

존재하지 않는 내용에 대한 환각이 발생한 모습입니다.

## LLM Tool 설정하기

Tool은 LLM이 사용할 수 있는 다양한 외부 기능을 의미합니다.   
LangChain에서 자체적으로 지원하는 Tool을 사용하거나, RAG의 Retriever를 Tool로 변환하는 것도 가능합니다.     
또한, 함수를 Tool로 변환할 수도 있습니다.

1. Tavily Search (http://app.tavily.com/)

Tavily는 AI 기반의 검색 엔진입니다. 계정별 월 1000개의 무료 사용량을 지원합니다.      
Tavily Search는 URL과 함께 내용의 간단한 요약을 지원하는 것이 특징입니다.


In [ ]:
os.environ['TAVILY_API_KEY'] = ''


In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    )
tavily_search

In [ ]:
search_docs = tavily_search.invoke("랭그래프와 랭체인의 차이")
search_docs

2.

# LLM에 Tool 연결하기   

생성한 툴은 llm.bind_tools()를 통해 LLM에 연결할 수 있습니다.    
이 기능은 Gemini, GPT, Claude 등의 툴에서 지원하는데요.     
HuggingFace 공개 모델에서 Tool을 사용하는 경우는 이후 실습에서 자세히 알아보겠습니다!

In [ ]:
tools = [tavily_search]

llm_with_tools = llm.bind_tools(tools)

llm_with_tools

랭체인에서, tool 정보는 tools에 저장되는데요.   
해당 내용은 랭체인 내부에서 json Schema 형식으로 프롬프트에 포함됩니다.

In [ ]:
# 시스템 메시지에 포함되는 내용
# (`LLM과 Agent` 강의 슬라이드를 참고하세요!)

llm_with_tools.kwargs['tools']

LLM은 프롬프트로 주어지는 툴 정보를 바탕으로 Tool의 사용을 결정합니다.    
Schema를 통해, 툴의 의미와 사용 방법, 형식을 이해합니다.

이후, LLM은 Tool 사용이 필요한 경우 특별한 포맷의 메시지를 출력합니다.   
이를 Tool Call Message라고 부릅니다.

In [ ]:
tool_chain = prompt | llm_with_tools
tool_call_msg = tool_chain.invoke("패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?")
tool_call_msg


In [ ]:
tool_call_msg = llm_with_tools.invoke("패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정인지 검색해서 알려줘")
tool_call_msg

In [ ]:
tool_call_msg.tool_calls
# 리스트 형식인 이유는? 여러 개의 툴을 동시에 사용할 수 있기 때문

`tool_calls`에 포함된 name을 활용하면 툴 결과를 전달할 수 있습니다.   
`name` 값은 문자열이므로, dictionary를 통해 연결합니다.

In [ ]:
tool_list ={'tavily_search_results_json': tavily_search}

tool_name = tool_call_msg.tool_calls[0]['name']
tool_exec = tool_list[tool_name]

tool_name, tool_exec

Tool에 tool_call을 입력해 invoke를 수행합니다.

In [ ]:
tool_msg = tool_exec.invoke(tool_call_msg.tool_calls[0])
tool_msg

이번에는 ToolMessage라는 새로운 형태의 메시지가 생성되었습니다!

툴 실행 결과를 LLM에게 다시 전달하여, 이를 해석하고 답변하게 만들어 보겠습니다.   

Chain 형식의 구성은 메시지를 변화시키는 것이 복잡하므로, 이전에 사용했던 `HumanMessage`에서 시작하여   
LLM의 답변인 `AIMessage`와 `ToolMessage` 를 추가합니다.

In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="패스트캠퍼스의 '랭그래프로 한번에 완성하는 복잡한 RAG와 Agent 과정'은 어떤 과정이야?")]

messages.append(tool_call_msg)
messages.append(tool_msg)

messages

In [ ]:
print(messages[2].content)

`Query-Tool Call-Tool`의 형식은 가장 기본적인 툴 사용 방법입니다.

In [ ]:
result = llm_with_tools.invoke(messages)
# 검색 결과를 바탕으로 답변한 모습

print(result.content)


이제, 지금까지 배운 내용을 종합하여 실행해 보겠습니다.   
질문이 주어지면, 질문에 대한 검색을 수행하여 결과를 바탕으로 답변합니다.   

<br><br>

그런데, 이번에는 분기가 필요합니다.   
검색이 필요없는 질문이라면, LLM은 tool을 요청하지 않을 것입니다.

In [ ]:
# LLM, Question, Tool을 이용한 간단한 함수
def simple_workflow(llm, question , tools = [tavily_search]):

    # 툴과 LLM 구성

    tool_list = {x.name: x for x in tools}
    # tavily_search.name = 'tavily_search_results_json' 을 이용하면
    # tool_list = {'tavily_search_results_json': tavily_search} 와 동일합니다.

    llm_with_tools = llm.bind_tools(tools)


    # 메시지 구성
    messages = [HumanMessage(content=question)]
    print('Query:', question)


    # LLM에 메시지 전달 (분기)
    tool_msg = llm_with_tools.invoke(question)
    messages.append(tool_msg)

    if tool_msg.tool_calls:
        # 툴 호출이 있을 경우: 툴 실행 후 결과를 전달
        tool_name = tool_msg.tool_calls[0]['name']

        print(f"-- {tool_name} 사용 중 --")
        print(tool_msg.tool_calls[0])


        tool_exec = tool_list[tool_name]

        tool_result = tool_exec.invoke(tool_msg.tool_calls[0])
        messages.append(tool_result)
        result = llm_with_tools.invoke(messages)

    else:
        # 툴 호출이 없을 경우: 처음 출력을 그대로 전달
        result = tool_msg

    return result.content

In [ ]:
response = simple_workflow(llm, "2025년 1월 출시된 딥시크의 언어 모델 이름이 뭐야?")
print(response)

# Custom Tool 만들기

Tavily Search의 경우, 랭체인에서 사전에 구성한 기본 Schema가 존재하지만,   

Tool을 새로 만드는 경우에는 description과 같은 값을 직접 구성해야 합니다.   
가장 간단한 방식은 랭체인의 Tool 데코레이터를 사용하는 것입니다.  

In [ ]:
from langchain_core.tools import tool

@tool
def multiply(x:int, y:int) -> int:
    "x와 y를 입력받아, x와 y를 곱한 결과를 반환합니다."
    return x*y

@tool
def current_date() -> str:
    "현재 날짜를 %y-%m-%d 형식으로 반환합니다."
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

print(multiply.invoke({'x':3, 'y':4}))
print(current_date.invoke({}))


In [ ]:
tools = [tavily_search, multiply, current_date]

In [ ]:
llm.bind_tools(tools)

In [ ]:
question = '오늘 날짜는?'

result = simple_workflow(llm, question, tools)
result


In [ ]:
# LLM은 오늘 날짜를 모른다
llm.invoke('오늘 날짜는?').content

<br><br>
## [연습문제] 복권 숫자 예측시키기

아래의 함수에 대해, 함수의 설명을 추가하여 llm의 툴로 전달해 봅시다.

In [ ]:
from typing import List
import random

@tool
def generate_random_numbers(min_val: int, max_val: int, count: int) -> List[int]:
    """주어진 범위 내에서 지정된 개수만큼의 중복되지 않은 랜덤 정수를 생성합니다.
    Args:
        min_val (int): 최솟값
        max_val (int): 최댓값
        count (int): 생성할 숫자의 개수

    Returns:
        List[int]: 중복되지 않은 랜덤 정수들의 리스트
    """
    # possible_range = max_val - min_val + 1
    # if count > possible_range:
    #     raise ValueError(f"생성 가능한 숫자의 범위({possible_range}개)보다 더 많은 숫자({count}개)를 요청했습니다.")

    return str(random.sample(range(min_val, max_val + 1), count))

In [ ]:
# LLM에 함수 Bind

llm_lotto = llm.bind_tools([generate_random_numbers])
llm_lotto.invoke('로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.')

In [ ]:
# simple_workflow 함수를 이용하여 실행하기

simple_workflow(llm_lotto, '로또 번호 추첨해줘! 1부터 45까지 6개를 뽑으면 돼.', [generate_random_numbers])

우리가 만든 `simple_workflow` 함수는 가장 간단한 형태의 Single Tool Calling을 수행하는 방식이었는데요.   

실제 환경의 시나리오는 훨씬 복잡합니다.   
한 번에 여러 개의 툴을 실행하거나, 툴 실행 결과를 받아 다음 툴에 활용할 수도 있을 것입니다.   

랭체인에서도 이와 같이 복잡한 작업을 수행하기 위한 기능이 자체적으로 구현되어 있는데요.    

예시로 `Structured Chat Agent`를 간단하게 만들고 실행해 보겠습니다.

# Agent

생각-행동-관찰을 거치는 ReAct 에이전트는 복잡한 플로우를 효과적으로 처리합니다.   
- 생각(Thought): 주어진 Context 상에서 다음 작업을 어떻게 수행할지 설명하는 과정
- 행동(Action): Tool 실행 명령어를 생성하는 과정
- 관찰(Thought): Tool 결과를 Context에 추가하는 과정


이 과정은 랭체인에 구성된 `create_structured_chat_agent`를 이용해 구성할 수 있습니다.

In [ ]:
from langchain.agents import AgentExecutor, create_structured_chat_agent
from langchain.prompts import ChatPromptTemplate

agent_prompt = ChatPromptTemplate([
    ('system','''
최대한 정확히 질문에 답변하세요. 당신은 다음의 툴을 사용할 수 있습니다:
{tools}

action 키 (tool name)와 action_input 키를 포함하는 json 형태로 출력하세요.

action의 값은 "Final Answer" 또는 {tool_names} 중 하나여야 합니다.
반드시 하나의 json 형태만 출력하세요. 다음은 예시입니다.
```
{{

  "action": $TOOL_NAME,

  "action_input": $INPUT

}}
```

아래의 포맷으로 답변하세요.:

Question: 최종적으로 답변해야 하는 질문
Thought: 무엇을 해야 하는지를 항상 떠올리세요.
Action:
```
$JSON_BLOB
```
Observation: 액션의 실행 결과
... (이 Thought/Action/Observation 은 10번 이내로 반복될 수 있습니다.)

Thought: 이제 답을 알겠다!
Action:
```
{{
  "action": "Final Answer",
  "action_input": "Final response to human"
}}
```
'''),
('user','''Question: {input}
Thought: {agent_scratchpad}''')])


agent = create_structured_chat_agent(llm, tools, agent_prompt)

In [ ]:
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

agent_executor.invoke({'input':"Open Source Multimodal LLM Model 추천해줘"})

In [ ]:
agent_executor.invoke({'input':"레오나르도 디카프리오의 출생년도를 찾은 뒤, 각 숫자를 순서대로 곱해줘."})

랭체인의 에이전트 구조는 매우 간결하지만, 컨트롤하기가 매우 어려운데요.    
실제로 우리가 개발하는 구조에서는, 모든 단계에서 LLM이 컨텍스트를 저장하여 판단을 내릴 필요는 없습니다.   
또한, 모든 과정이 자연어로 구성되기 때문에 중간에 파싱 오류가 발생하는 경우에 동작이 멈추기도 합니다.   


특정 작업을 반복 실행하거나, 정해진 순서에 따라 실행해야 하는 상황이라면 어떻게 해야 할까요?    
랭체인으로 이와 같은 기능을 구현하는 것도 가능하지만, 다소 복잡한데요.   


랭그래프를 이용하면 구체적인 Workflow를 바탕으로 Agent 형태의 어플리케이션을 효과적으로 만들 수 있습니다.   
